In [63]:
from datetime import datetime
print(str(datetime.now()))
import numpy as np
# os.environ["TF_CPP_MIN_LOG_LEVEL"] ="3"
import tensorflow as tf
import tensorflow_recommenders as tfrs
import pandas as pd
import math

import os
import glob

2022-08-23 22:56:13.979795


In [5]:
gpu_available = tf.test.is_gpu_available()
gpu_available
req_cols = ['ITEM_ID', 'USER_ID', 'CABIN_TYPE', 'USER_RESIDENCE']

In [18]:
data_set_df_updated = pd.read_csv("interaction_demo.csv")
test_df = pd.read_csv("interaction_test_demo.csv")
data_set_df_updated.loc[data_set_df_updated.USER_RESIDENCE.isnull(),"USER_RESIDENCE"] = 'None'
test_df.loc[test_df.USER_RESIDENCE.isnull(),"USER_RESIDENCE"] = 'None'
train_df = pd.concat([data_set_df_updated, test_df], ignore_index=True)
train_df.sort_values("TIMESTAMP", ascending= False, inplace= True)

In [22]:
item_df = train_df[["ITEM_ID"]].drop_duplicates("ITEM_ID")
item_ds = tf.data.Dataset.from_tensor_slices(item_df.to_dict("list")).batch(32)
item_df.shape

(350, 1)

In [23]:
train_ds =  tf.data.Dataset.from_tensor_slices(train_df[req_cols].to_dict("list")).batch(256)

In [24]:
USER_ID_unique = np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["USER_ID"]))))

CABIN_TYPE_unique = np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["CABIN_TYPE"]))))

USER_RESIDENCE_unique = np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["USER_RESIDENCE"]))))

In [26]:
item_unique =  np.unique(np.concatenate(list(train_ds.map(
        lambda x: x["ITEM_ID"]))))

In [27]:
class UserModel(tf.keras.Model):

    def __init__(self):
        super().__init__()

        emb_dim = 32    
        self.user_id_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=USER_ID_unique, mask_token=None),
            tf.keras.layers.Embedding(len(USER_ID_unique) + 1, emb_dim),
        ])
            
        self.cabin_type_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary= CABIN_TYPE_unique, mask_token=None),  
            tf.keras.layers.Embedding(len(CABIN_TYPE_unique) + 1, emb_dim),
        ])

        self.user_residence_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=USER_RESIDENCE_unique, mask_token=None),
            tf.keras.layers.Embedding(len(USER_RESIDENCE_unique) + 1, emb_dim),
        ])
        

    def call(self, user_interation_data):
        return tf.concat([                          
            self.user_id_embedding(user_interation_data["USER_ID"]), 
            self.cabin_type_embedding(user_interation_data["CABIN_TYPE"]), 
            self.user_residence_embedding(user_interation_data["USER_RESIDENCE"]),
        ], axis=1)





In [28]:
class ItemModel(tf.keras.Model):

    def __init__(self):
        super().__init__()

        

        self.item_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=item_unique, mask_token=None),
            tf.keras.layers.Embedding(len(item_unique) + 1, 32),
        ])


    def call(self, user_interation_data):

        return tf.concat([
            self.item_embedding(user_interation_data["ITEM_ID"])
            
            ], axis=1)

In [88]:
class TRFSRetrievalModel(tfrs.models.Model):

    def __init__(self, UserModel,ItemModel, item_ds ):
        super().__init__()



        self.query_model = tf.keras.Sequential([#,UserModel()
          UserModel(),
          tf.keras.layers.Dense(32 , kernel_initializer= tf.keras.initializers.RandomNormal(seed=99)),   
        ])
        

        self.candidate_model = tf.keras.Sequential([
          ItemModel(),
          tf.keras.layers.Dense(32, kernel_initializer= tf.keras.initializers.RandomNormal(seed=1))
        ]) 
        
        


#         metrics = [
#           tf.keras.metrics.TopKCategoricalAccuracy(
#               k=x, name=f"factorized_top_k/top_{x}_categorical_accuracy")
#           for x in [3,5,10,15, 25]
#         ]  
        
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
            item_ds.map(self.candidate_model),
                ks= (3, 5, 10,15, 25))
        )
        
#         self.task = tfrs.tasks.Retrieval(
#             metrics=tfrs.metrics.FactorizedTopK(
#                 candidates=item_ds.map(self.candidate_model),
#                 metrics = metrics,
#                 k = 100
#             ),
#             # temperature = 0.5,
#             num_hard_negatives = 5
#         )

    def compute_loss(self, features, training= True):

        item_features = {"ITEM_ID":features.pop("ITEM_ID") }
        query_embeddings = self.query_model(features)
        item_embeddings = self.candidate_model(item_features)

        return self.task(query_embeddings, 
        item_embeddings, 
        compute_metrics=True
        )

    def call(self, test):
        features= test.copy()
        item_features = {"ITEM_ID":features.pop("ITEM_ID") }
        query_embeddings = self.query_model(features)
        item_embeddings = self.candidate_model(item_features)

        return query_embeddings, item_embeddings


In [89]:
test_max_index = math.floor(train_df.shape[0]*0.1)
train_split_len = train_df.shape[0] - test_max_index
data_set_tf = tf.data.Dataset.from_tensor_slices(train_df[req_cols].to_dict("list"))
test = data_set_tf.take(test_max_index)
train = data_set_tf.skip(test_max_index).take(train_split_len)
shuffled = train.shuffle(train_split_len, seed=42, reshuffle_each_iteration=True)
cached_train = shuffled.batch(512).prefetch(4096)#train.shuffle(100_000).batch(2048)
cached_test = test.batch(512).cache()


In [90]:
train_df.shape[0], test_max_index

(34853, 3485)

In [91]:
def delete_all_check_points(fpath= 'new_amazon_check_points/*'):
    files = glob.glob(fpath)
    for f in files:
        os.remove(f)
        

In [92]:
def get_call_back_fun(K):
    delete_all_check_points()
    model_path = f"new_amazon_check_points/best_check_point_{K}k"
    
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=model_path,
        save_weights_only=True,
        monitor=f'val_factorized_top_k/top_{K}_categorical_accuracy',
        mode='max',
        save_best_only=True)

    early_stoping = tf.keras.callbacks.EarlyStopping(monitor=f'val_factorized_top_k/top_{K}_categorical_accuracy',
                                                     mode='min',
                                                     patience=5)

    reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor=f'val_factorized_top_k/top_{K}_categorical_accuracy', 
                                                     factor=0.6,
                                                     #mode='min',
                                                     patience=9, 
                                                     min_lr=1e-6
    )
    return model_path, model_checkpoint_callback, early_stoping, reduce_lr

In [93]:
model_path, model_checkpoint_callback, early_stoping, reduce_lr = get_call_back_fun(10)

In [94]:
model = TRFSRetrievalModel(UserModel, ItemModel, item_ds)
model.compile(optimizer=tf.keras.optimizers.Adam(0.01)) 

In [95]:
model.fit(cached_train,
          validation_data=cached_test,
          epochs=100,
          verbose=1, 
          workers=3,
          use_multiprocessing=True,
          callbacks=[model_checkpoint_callback, 
                     reduce_lr]

          )

Epoch 1/100
62/62 [==============================] - 4s 55ms/step - factorized_top_k/top_3_categorical_accuracy: 0.0607 - factorized_top_k/top_5_categorical_accuracy: 0.1112 - factorized_top_k/top_10_categorical_accuracy: 0.2122 - factorized_top_k/top_15_categorical_accuracy: 0.2784 - factorized_top_k/top_25_categorical_accuracy: 0.3744 - loss: 2944.3048 - regularization_loss: 0.0000e+00 - total_loss: 2944.3048 - val_factorized_top_k/top_3_categorical_accuracy: 0.0875 - val_factorized_top_k/top_5_categorical_accuracy: 0.1271 - val_factorized_top_k/top_10_categorical_accuracy: 0.2729 - val_factorized_top_k/top_15_categorical_accuracy: 0.3257 - val_factorized_top_k/top_25_categorical_accuracy: 0.4106 - val_loss: 2259.5398 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2259.5398 - lr: 0.0100
Epoch 2/100
62/62 [==============================] - 3s 51ms/step - factorized_top_k/top_3_categorical_accuracy: 0.2120 - factorized_top_k/top_5_categorical_accuracy: 0.3091 - factorized_top_

62/62 [==============================] - 3s 49ms/step - factorized_top_k/top_3_categorical_accuracy: 0.7215 - factorized_top_k/top_5_categorical_accuracy: 0.7609 - factorized_top_k/top_10_categorical_accuracy: 0.8153 - factorized_top_k/top_15_categorical_accuracy: 0.8450 - factorized_top_k/top_25_categorical_accuracy: 0.8830 - loss: 1351.3477 - regularization_loss: 0.0000e+00 - total_loss: 1351.3477 - val_factorized_top_k/top_3_categorical_accuracy: 0.1102 - val_factorized_top_k/top_5_categorical_accuracy: 0.1478 - val_factorized_top_k/top_10_categorical_accuracy: 0.2095 - val_factorized_top_k/top_15_categorical_accuracy: 0.2680 - val_factorized_top_k/top_25_categorical_accuracy: 0.3489 - val_loss: 2625.7820 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2625.7820 - lr: 0.0100
Epoch 9/100
62/62 [==============================] - 3s 51ms/step - factorized_top_k/top_3_categorical_accuracy: 0.7332 - factorized_top_k/top_5_categorical_accuracy: 0.7750 - factorized_top_k/top_10_cat

62/62 [==============================] - 3s 50ms/step - factorized_top_k/top_3_categorical_accuracy: 0.7941 - factorized_top_k/top_5_categorical_accuracy: 0.8457 - factorized_top_k/top_10_categorical_accuracy: 0.9006 - factorized_top_k/top_15_categorical_accuracy: 0.9250 - factorized_top_k/top_25_categorical_accuracy: 0.9495 - loss: 1155.1792 - regularization_loss: 0.0000e+00 - total_loss: 1155.1792 - val_factorized_top_k/top_3_categorical_accuracy: 0.1484 - val_factorized_top_k/top_5_categorical_accuracy: 0.2006 - val_factorized_top_k/top_10_categorical_accuracy: 0.2778 - val_factorized_top_k/top_15_categorical_accuracy: 0.3346 - val_factorized_top_k/top_25_categorical_accuracy: 0.4129 - val_loss: 2742.9885 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2742.9885 - lr: 0.0060
Epoch 19/100
62/62 [==============================] - 3s 52ms/step - factorized_top_k/top_3_categorical_accuracy: 0.7986 - factorized_top_k/top_5_categorical_accuracy: 0.8531 - factorized_top_k/top_10_ca

62/62 [==============================] - 3s 49ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8233 - factorized_top_k/top_5_categorical_accuracy: 0.8790 - factorized_top_k/top_10_categorical_accuracy: 0.9277 - factorized_top_k/top_15_categorical_accuracy: 0.9479 - factorized_top_k/top_25_categorical_accuracy: 0.9656 - loss: 1104.5741 - regularization_loss: 0.0000e+00 - total_loss: 1104.5741 - val_factorized_top_k/top_3_categorical_accuracy: 0.1572 - val_factorized_top_k/top_5_categorical_accuracy: 0.2149 - val_factorized_top_k/top_10_categorical_accuracy: 0.2996 - val_factorized_top_k/top_15_categorical_accuracy: 0.3492 - val_factorized_top_k/top_25_categorical_accuracy: 0.4209 - val_loss: 2753.9443 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2753.9443 - lr: 0.0060
Epoch 29/100
62/62 [==============================] - 3s 50ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8268 - factorized_top_k/top_5_categorical_accuracy: 0.8831 - factorized_top_k/top_10_ca

62/62 [==============================] - 3s 50ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8393 - factorized_top_k/top_5_categorical_accuracy: 0.8972 - factorized_top_k/top_10_categorical_accuracy: 0.9435 - factorized_top_k/top_15_categorical_accuracy: 0.9605 - factorized_top_k/top_25_categorical_accuracy: 0.9755 - loss: 1078.2730 - regularization_loss: 0.0000e+00 - total_loss: 1078.2730 - val_factorized_top_k/top_3_categorical_accuracy: 0.1693 - val_factorized_top_k/top_5_categorical_accuracy: 0.2241 - val_factorized_top_k/top_10_categorical_accuracy: 0.3044 - val_factorized_top_k/top_15_categorical_accuracy: 0.3524 - val_factorized_top_k/top_25_categorical_accuracy: 0.4273 - val_loss: 2794.6125 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2794.6125 - lr: 0.0060
Epoch 39/100
62/62 [==============================] - 3s 51ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8404 - factorized_top_k/top_5_categorical_accuracy: 0.8991 - factorized_top_k/top_10_ca

...

62/62 [==============================] - 3s 53ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8551 - factorized_top_k/top_5_categorical_accuracy: 0.9138 - factorized_top_k/top_10_categorical_accuracy: 0.9572 - factorized_top_k/top_15_categorical_accuracy: 0.9723 - factorized_top_k/top_25_categorical_accuracy: 0.9842 - loss: 1050.8590 - regularization_loss: 0.0000e+00 - total_loss: 1050.8590 - val_factorized_top_k/top_3_categorical_accuracy: 0.1802 - val_factorized_top_k/top_5_categorical_accuracy: 0.2407 - val_factorized_top_k/top_10_categorical_accuracy: 0.3188 - val_factorized_top_k/top_15_categorical_accuracy: 0.3742 - val_factorized_top_k/top_25_categorical_accuracy: 0.4516 - val_loss: 2776.5068 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2776.5068 - lr: 0.0060
Epoch 59/100
62/62 [==============================] - 3s 51ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8551 - factorized_top_k/top_5_categorical_accuracy: 0.9124 - factorized_top_k/top_10_ca

62/62 [==============================] - 3s 50ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8580 - factorized_top_k/top_5_categorical_accuracy: 0.9195 - factorized_top_k/top_10_categorical_accuracy: 0.9613 - factorized_top_k/top_15_categorical_accuracy: 0.9753 - factorized_top_k/top_25_categorical_accuracy: 0.9862 - loss: 1043.1312 - regularization_loss: 0.0000e+00 - total_loss: 1043.1312 - val_factorized_top_k/top_3_categorical_accuracy: 0.1877 - val_factorized_top_k/top_5_categorical_accuracy: 0.2494 - val_factorized_top_k/top_10_categorical_accuracy: 0.3182 - val_factorized_top_k/top_15_categorical_accuracy: 0.3707 - val_factorized_top_k/top_25_categorical_accuracy: 0.4548 - val_loss: 2806.0564 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2806.0564 - lr: 0.0060
Epoch 69/100
62/62 [==============================] - 3s 50ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8593 - factorized_top_k/top_5_categorical_accuracy: 0.9192 - factorized_top_k/top_10_ca

62/62 [==============================] - 3s 55ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8718 - factorized_top_k/top_5_categorical_accuracy: 0.9324 - factorized_top_k/top_10_categorical_accuracy: 0.9698 - factorized_top_k/top_15_categorical_accuracy: 0.9812 - factorized_top_k/top_25_categorical_accuracy: 0.9906 - loss: 1011.8246 - regularization_loss: 0.0000e+00 - total_loss: 1011.8246 - val_factorized_top_k/top_3_categorical_accuracy: 0.1888 - val_factorized_top_k/top_5_categorical_accuracy: 0.2571 - val_factorized_top_k/top_10_categorical_accuracy: 0.3349 - val_factorized_top_k/top_15_categorical_accuracy: 0.3793 - val_factorized_top_k/top_25_categorical_accuracy: 0.4651 - val_loss: 2950.7559 - val_regularization_loss: 0.0000e+00 - val_total_loss: 2950.7559 - lr: 0.0036
Epoch 79/100
62/62 [==============================] - 3s 52ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8727 - factorized_top_k/top_5_categorical_accuracy: 0.9314 - factorized_top_k/top_10_ca

62/62 [==============================] - 3s 50ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8797 - factorized_top_k/top_5_categorical_accuracy: 0.9394 - factorized_top_k/top_10_categorical_accuracy: 0.9754 - factorized_top_k/top_15_categorical_accuracy: 0.9853 - factorized_top_k/top_25_categorical_accuracy: 0.9927 - loss: 990.8615 - regularization_loss: 0.0000e+00 - total_loss: 990.8615 - val_factorized_top_k/top_3_categorical_accuracy: 0.2023 - val_factorized_top_k/top_5_categorical_accuracy: 0.2611 - val_factorized_top_k/top_10_categorical_accuracy: 0.3395 - val_factorized_top_k/top_15_categorical_accuracy: 0.3894 - val_factorized_top_k/top_25_categorical_accuracy: 0.4746 - val_loss: 3110.5818 - val_regularization_loss: 0.0000e+00 - val_total_loss: 3110.5818 - lr: 0.0022
Epoch 89/100
62/62 [==============================] - 3s 50ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8786 - factorized_top_k/top_5_categorical_accuracy: 0.9398 - factorized_top_k/top_10_cate

62/62 [==============================] - 3s 50ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8795 - factorized_top_k/top_5_categorical_accuracy: 0.9418 - factorized_top_k/top_10_categorical_accuracy: 0.9771 - factorized_top_k/top_15_categorical_accuracy: 0.9872 - factorized_top_k/top_25_categorical_accuracy: 0.9938 - loss: 991.7849 - regularization_loss: 0.0000e+00 - total_loss: 991.7849 - val_factorized_top_k/top_3_categorical_accuracy: 0.2029 - val_factorized_top_k/top_5_categorical_accuracy: 0.2534 - val_factorized_top_k/top_10_categorical_accuracy: 0.3366 - val_factorized_top_k/top_15_categorical_accuracy: 0.3874 - val_factorized_top_k/top_25_categorical_accuracy: 0.4654 - val_loss: 3156.9324 - val_regularization_loss: 0.0000e+00 - val_total_loss: 3156.9324 - lr: 0.0022
Epoch 99/100
62/62 [==============================] - 3s 50ms/step - factorized_top_k/top_3_categorical_accuracy: 0.8799 - factorized_top_k/top_5_categorical_accuracy: 0.9413 - factorized_top_k/top_10_cate

In [96]:
model.load_weights("new_amazon_check_points/best_check_point_10k")

In [97]:
result_summary = model.evaluate(cached_test)

7/7 [==============================] - 0s 41ms/step - factorized_top_k/top_3_categorical_accuracy: 0.2046 - factorized_top_k/top_5_categorical_accuracy: 0.2623 - factorized_top_k/top_10_categorical_accuracy: 0.3432 - factorized_top_k/top_15_categorical_accuracy: 0.3986 - factorized_top_k/top_25_categorical_accuracy: 0.4838 - loss: 3742.8446 - regularization_loss: 0.0000e+00 - total_loss: 3742.8446


In [99]:
{val:result_summary[idx] for idx, val in enumerate([3, 5, 10,15, 25]) }

{3: 0.20459111034870148,
 5: 0.2622668445110321,
 10: 0.34318506717681885,
 15: 0.39856529235839844,
 25: 0.4837876558303833}